# Лабораторная 1. Кобылкин Константин. Вариант 39

In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [3]:
from pyspark import SparkContext, SparkConf

config = SparkConf()
config.set("spark.app.name", "KKS lab1")

sc = SparkContext(conf=config)

# Загрузка данных

In [4]:
ratings = sc.textFile("/labs/laba01/ml-100k/u.data")

In [10]:
def getRatingsDistributionSorted(ratings_distribution_tuples):
    return OrderedDict(sorted(dict(ratings_distribution_tuples).items()))

# Распределение оценок по всем фильмам

In [5]:
all_ratings_distribution = (ratings.map(lambda x: x.split('\t'))
                            .map(lambda x: (int(x[2]), 1))
                            .reduceByKey(lambda x, y: x + y)
                            .collect())

In [6]:
from collections import OrderedDict

In [13]:
all_ratings_distribution_dict = getRatingsDistributionSorted(all_ratings_distribution)

# Распределение оценок по заданному фильму

In [8]:
movie_id = 328

In [9]:
specific_movie_ratings_distribution = (ratings.map(lambda x: x.split('\t'))
                                       .filter(lambda x: int(x[1]) == movie_id)
                                       .map(lambda x: (int(x[2]), 1))
                                       .reduceByKey(lambda x, y: x + y)
                                       .collect())

In [14]:
specific_movie_ratings_distribution_dict = getRatingsDistributionSorted(specific_movie_ratings_distribution)

# Формирование финального результата и запись в файл

In [17]:
final_result = OrderedDict((("hist_film", list(specific_movie_ratings_distribution_dict.values())),
                            ("hist_all", list(all_ratings_distribution_dict.values()))))

In [18]:
final_result

OrderedDict([('hist_film', [12, 40, 94, 109, 40]),
             ('hist_all', [6110, 11370, 27145, 34174, 21201])])

In [20]:
import json

In [21]:
with open('lab01.json', 'w') as fp:
    json.dump(final_result, fp) 

In [ ]:
sc.stop()